In [2]:
import torch
import torchvision 
import torch.nn as nn 
import torch.nn.functional as F 
from torch.utils.data import DataLoader 
import torchtext
from torchvision.transforms import transforms
import matplotlib.pyplot as plt
import numpy as np 


In [3]:
train_data = torchvision.datasets.FashionMNIST(root='zzzz', transform = transforms.ToTensor())
train_data[0][0].shape

torch.Size([1, 28, 28])

In [4]:
dataloader = DataLoader(train_data, batch_size=32, shuffle=True)

In [5]:
data, sample = load.__iter__().next()

NameError: name 'load' is not defined

In [37]:
sample

tensor([0, 4, 2, 0, 0, 9, 1, 4, 8, 4, 3, 5, 4, 6, 6, 3, 5, 2, 6, 0, 8, 2, 7, 9,
        2, 2, 5, 2, 9, 9, 3, 6])

In [43]:
from resnet import Resnet, Residual
from preact_resnet import PreActBlock, PreActResNet

model = PreActResNet(PreActBlock, [2,2,2,2])
device = 'cuda'
model = model.to(device)
epochs = 3 
iter = 1
p_iter = 200
learning_rate = 1e-4
optimizer = torch.optim.Adam(params=model.parameters(), lr= learning_rate)
criterion = nn.CrossEntropyLoss()
loss_list = []
acc_list = []
total_loss = 0
model.train()
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[500,1000,1500], gamma=0.5)

for epoch in range(epochs):
    for sample, labels in dataloader: 
        sample, labels = sample.to(device), labels.to(device)
        optimizer.zero_grad()
        pred = model(sample)
        loss = criterion(pred, labels)
        loss.backward()
        total_loss += loss.item()
        optimizer.step()
        scheduler.step()

        if iter % p_iter == 0:
            pred_ = torch.argmax(pred, dim=1)
            correct = pred_.eq(labels)
            acc = torch.mean(correct.float())
            loss_list.append(total_loss/p_iter)
            acc_list.append(acc)
            print('Epoch{}/{}, iteration{} -> loss{:.4f}, acc:{:.3f}'.format(epoch+1, epochs, iter, total_loss/p_iter, acc))
            total_loss = 0
            
        iter += 1 

plt.plot(loss_list, label='loss')
y = []
for i in acc_list: 
    i = i.cpu().data.numpy()
    y.append(i)

plt.plot(y, label='acc')
plt.legend()
plt.show()


RuntimeError: Given groups=1, weight of size [64, 3, 3, 3], expected input[32, 1, 28, 28] to have 3 channels, but got 1 channels instead